In [1]:
 pip install sklearn-crfsuite

     |████████████████████████████████| 747kB 4.9MB/s 


In [3]:
pip install conllu

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
import pandas as pd

In [6]:
dataset = pd.read_csv('/content/drive/My Drive/CS60075-NLP/NLP Assignment-4/hi-ud-train.conllu')


In [7]:

dataset.head()

,ID,WORD,POS_TAG
0,1.0,yaha,DET
1,2.0,eSiyA,PROPN
2,3.0,kI,ADP
3,4.0,sabase,ADV
4,5.0,badZI,ADJ


In [8]:
test_file = open('/content/drive/My Drive/CS60075-NLP/NLP Assignment-4/hi-ud-test .conllu', 'r') 

In [9]:
test_data = test_file.readlines()
test_lines = list()
test_set = list()
for line in test_data:
  lin = tuple((line.strip()).split('\t'))
  if lin[0]!='':
    test_lines.append(lin)
  if lin[0]!='' and lin[1]== ".":
    test_set.append(test_lines)
    test_lines = []


In [10]:
test_set[0]

[('ID', 'WORD', 'TAG'),
 ('1', 'rAmAyaNa', 'PROPN'),
 ('2', 'kAla', 'PROPN'),
 ('3', 'meM', 'ADP'),
 ('4', 'BagavAna', 'NOUN'),
 ('5', 'rAma', 'PROPN'),
 ('6', 'ke', 'ADP'),
 ('7', 'puwra', 'NOUN'),
 ('8', 'kuSa', 'PROPN'),
 ('9', 'kI', 'ADP'),
 ('10', 'rAjaXAnI', 'NOUN'),
 ('11', 'kuSAvawI', 'PROPN'),
 ('12', 'ko', 'ADP'),
 ('13', '483', 'PROPN'),
 ('14', 'IsA', 'PROPN'),
 ('15', 'pUrva', 'PROPN'),
 ('16', 'buxXa', 'PROPN'),
 ('17', 'ne', 'ADP'),
 ('18', 'apane', 'PRON'),
 ('19', 'aMwima', 'ADJ'),
 ('20', 'viSrAma', 'NOUN'),
 ('21', 'ke', 'ADP'),
 ('22', 'lie', 'ADP'),
 ('23', 'cunA', 'VERB'),
 ('24', '.', 'PUNCT')]

In [11]:
dataset = dataset.dropna()
dataset= dataset.reset_index()

In [12]:
train_lines = []
train_set = []
for i in range(len(dataset)):
  id = str(dataset['ID'][i])
  word = dataset['WORD'][i]
  tag = dataset['POS_TAG'][i]
  train_lines.append((id,word,tag))
  if word ==".":
    train_set.append(train_lines[1:])
    train_lines=[]

In [13]:
train_set[1]

[('2.0', 'navAba', 'NOUN'),
 ('3.0', 'SAhajehana', 'PROPN'),
 ('4.0', 'ne', 'ADP'),
 ('5.0', 'banavAyA', 'VERB'),
 ('6.0', 'WA', 'AUX'),
 ('7.0', '.', 'PUNCT')]

In [15]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [16]:
def word2features(sent, i):
    word = sent[i][1]

    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),      
    }
    if i > 0:
        word1 = sent[i-1][1]
        postag1 = sent[i-1][2]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][1]
        postag1 = sent[i+1][2]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for id, token, label in sent]

def sent2tokens(sent):
    return [token for id,token, label in sent]

Feature description:

* Lowercasing (previous, current and next words)
* Checking for uppercase
* Checking for title
* Checking for digit
* Checking for later part of words(last 3/2 letters)


In [17]:
%%time
X_train = [sent2features(s) for s in train_set]
y_train = [sent2labels(s) for s in train_set]

X_test = [sent2features(s) for s in test_set]
y_test = [sent2labels(s) for s in test_set]

CPU times: user 21.6 ms, sys: 8 ms, total: 29.6 ms
Wall time: 31.5 ms


In [18]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [19]:
labels = list(crf.classes_)
# labels.remove('O')
labels

['PROPN',
 'ADP',
 'ADV',
 'ADJ',
 'NOUN',
 'NUM',
 'AUX',
 'PUNCT',
 'VERB',
 'CCONJ',
 'DET',
 'PRON',
 'PART',
 'SCONJ',
 'X']

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.8289603555900241

In [21]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           X      0.000     0.000     0.000         0
        PART      0.909     0.909     0.909        33
       CCONJ      1.000     1.000     1.000        25
       SCONJ      0.667     0.667     0.667         3
         ADJ      0.654     0.745     0.697        94
         ADP      0.949     0.971     0.960       309
         ADV      0.615     0.381     0.471        21
        VERB      0.826     0.768     0.796        99
         DET      0.838     0.861     0.849        36
        NOUN      0.743     0.888     0.809       329
        PRON      0.806     0.831     0.818        65
       PROPN      0.667     0.441     0.531       145
         NUM      0.957     0.880     0.917        25
       PUNCT      1.000     0.830     0.907       135
         AUX      0.915     0.935     0.925       139

   micro avg      0.833     0.834     0.834      1458
   macro avg      0.770     0.740     0.750      1458
weighted avg      0.834   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
VERB   -> AUX     3.844053
PROPN  -> PROPN   2.769828
ADJ    -> NOUN    2.235242
AUX    -> AUX     2.221800
AUX    -> SCONJ   2.138434
PROPN  -> ADP     2.065055
VERB   -> SCONJ   2.061378
NUM    -> NOUN    1.749800
PROPN  -> CCONJ   1.654623
DET    -> ADJ     1.643873
AUX    -> CCONJ   1.556993
PROPN  -> PUNCT   1.540174
PART   -> NUM     1.509762
DET    -> NOUN    1.507429
NOUN   -> ADP     1.495227
PRON   -> ADP     1.417809
CCONJ  -> PROPN   1.276386
ADJ    -> VERB    1.172474
NUM    -> NUM     1.144962
ADV    -> VERB    1.123591

Top unlikely transitions:
PART   -> CCONJ   -0.908055
PROPN  -> DET     -0.931718
PRON   -> CCONJ   -0.937958
VERB   -> VERB    -0.947408
PRON   -> PUNCT   -0.976604
PROPN  -> PART    -1.005595
VERB   -> NOUN    -1.035852
DET    -> PROPN   -1.095875
DET    -> CCONJ   -1.123717
ADP    -> CCONJ   -1.124999
AUX    -> VERB    -1.179604
PROPN  -> AUX     -1.195544
ADV    -> PROPN   -1.219184
AUX    -> ADJ     -1.222223
ADV    -> AUX    

In [42]:
correct_pred = 0
allPred = 0
for i in range(len(y_pred)):
  for j in range(len(y_pred[i])):
    if(y_pred[i][j]==y_test[i][j]):
      correct_pred = correct_pred+1
    allPred = allPred+1
print("Overall test accuracy : ",correct_pred/allPred)


Overall test accuracy :  0.8334475668265936


In [44]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
y_pred1 = crf.predict(X_train)
print(metrics.flat_classification_report(
    y_train, y_pred1, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           X      1.000     1.000     1.000         2
        PART      1.000     0.994     0.997       160
       CCONJ      0.993     1.000     0.996       138
       SCONJ      0.980     1.000     0.990        49
         ADJ      0.998     0.998     0.998       539
         ADP      0.999     0.999     0.999      1387
         ADV      0.979     0.969     0.974        96
        VERB      0.998     0.979     0.989       634
         DET      1.000     1.000     1.000       157
        NOUN      0.992     0.997     0.995      1529
        PRON      1.000     0.996     0.998       278
       PROPN      0.997     0.987     0.992       611
         NUM      1.000     0.993     0.997       147
       PUNCT      1.000     1.000     1.000       564
         AUX      0.984     1.000     0.992       730

    accuracy                          0.995      7021
   macro avg      0.995     0.994     0.994      7021
weighted avg      0.995   

In [46]:
correct_pred = 0
allPred = 0
for i in range(len(y_pred1)):
  for j in range(len(y_pred1[i])):
    if(y_pred1[i][j]==y_train[i][j]):
      correct_pred = correct_pred+1
    allPred = allPred+1
print("Overall train accuracy : ",correct_pred/allPred)

Overall train accuracy :  0.9951573849878934
